In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import glob
import os

plt.style.use('ggplot') # Make the graphs a bit prettier

In [2]:
data = {}
files = glob.glob(os.path.expanduser("~/dev/ma/data/results/seq_louvain/*.json")) + glob.glob(os.path.expanduser("~/dev/ma/data/results/dlm_all/*.json")) + glob.glob(os.path.expanduser("~/dev/ma/data/results/plm_all/*.json"))

for path in files:
  for typename, items in json.load(open(path)).items():
    if typename in data:
      data[typename].update(items)
    else:
      data[typename] = items

frames = { typename: pd.DataFrame.from_dict(items, orient='index') for typename, items in data.items() }

algorithm_names = {
    'thrill partitioned louvain': 'partitioned local moving',
    'thrill node based fully distributed local moving': 'synchronous local moving'
}

frames['algorithm_run'].replace({ 'algorithm': algorithm_names }, inplace=True)

frames['program_run']['graph_path'] = frames['program_run']['graph']

graph_names = {
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_100000-sorted-preprocessed-*.bin': 'LFR 100K', 
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_1000000-preprocessed-*.bin': 'LFR 1M', 
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_10000000-preprocessed-*.bin': 'LFR 10M', 
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_100000000-preprocessed-*.bin': 'LFR 100M', 
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_200000000-sorted-preprocessed-*.bin': 'LFR 200M', 
    '/home/kit/iti/kp0036/graphs/hypercubegraph23-preprocessed-*.bin': 'hypercube', 
    '/home/kit/iti/kp0036/graphs/uk-2002.metis-preprocessed-*.bin': 'uk-2002', 
    '/home/kit/iti/kp0036/graphs/uk-2007-05.metis-preprocessed-*.bin': 'uk-2007-05', 
    '/home/kit/iti/kp0036/graphs/in-2004.metis-preprocessed-*.bin': 'in-2004', 
    '/home/kit/iti/kp0036/graphs/com-friendster-preprocessed-*.bin': 'com-friendster', 
    '/home/kit/iti/kp0036/graphs/com-lj.ungraph-preprocessed-*.bin': 'com-lj', 
    '/home/kit/iti/kp0036/graphs/com-orkut.ungraph-preprocessed-*.bin': 'com-orkut', 
    '/home/kit/iti/kp0036/graphs/com-youtube.ungraph-preprocessed-*.bin': 'com-youtube', 
    '/home/kit/iti/kp0036/graphs/com-amazon.ungraph-preprocessed-*.bin': 'com-amazon'
}

frames['program_run'].replace({ 'graph': graph_names }, inplace=True)

def calc_worker_label(x):
    if np.isnan(x['hosts']):
        return 'sequential'
    else: 
        return str(int(x['hosts'])) + 'x' + str(int(x['workers_per_host']))
        
frames['program_run']['worker_label'] = frames['program_run'].apply(calc_worker_label, axis=1)

In [9]:
frames['ground_truth'] = pd.DataFrame([{ 'graph': 'LFR 10M', 'modularity': 0.59928, 'cluster_count': 4542 },
{ 'graph': 'LFR 1M', 'modularity': 0.592803, 'cluster_count': 457 },
{ 'graph': 'LFR 100K', 'modularity': 0.524983, 'cluster_count': 49 },
{ 'graph': 'LFR 200M', 'modularity': 0.599963, 'cluster_count': 91636 },
{ 'graph': 'LFR 100M', 'modularity': 0.599928, 'cluster_count': 45653 }]).set_index('graph')


In [16]:
frames['clustering'] \
    .merge(frames['algorithm_run'], left_on='algorithm_run_id', right_index=True) \
    .merge(frames['program_run'], left_on='program_run_id', right_index=True) \
    .groupby(['algorithm', 'worker_label', 'graph'])[['runtime', 'modularity']].mean() \
    .stack() \
    .unstack(['algorithm', 'worker_label'])

algorithm                 partitioned local moving                         \
worker_label                                 16x16        2x16       4x16   
graph                                                                       
LFR 100K       runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
LFR 100M       runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
LFR 10M        runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
LFR 1M         runtime                         NaN         NaN  49.133987   
               modularity                      NaN         NaN   0.592719   
LFR 200M       runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
com-amazon     runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
com-friendster runtime                 1442.971659         NaN        NaN   
               modularity                 0.616655         NaN        NaN   
com-lj         runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
com-orkut      runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
com-youtube    runtime                         NaN  150.616430        NaN   
               modularity                      NaN    0.713472        NaN   
hypercube      runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
in-2004        runtime                         NaN         NaN  68.560918   
               modularity                      NaN         NaN   0.980307   
uk-2002        runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
uk-2007-05     runtime                 1185.680737         NaN        NaN   
               modularity                 0.996166         NaN        NaN   

algorithm                             sequential louvain  \
worker_label                     8x16         sequential   
graph                                                      
LFR 100K       runtime            NaN           1.438680   
               modularity         NaN           0.521836   
LFR 100M       runtime            NaN                NaN   
               modularity         NaN                NaN   
LFR 10M        runtime     225.192853        1018.980000   
               modularity    0.599273           0.599285   
LFR 1M         runtime            NaN          28.929400   
               modularity         NaN           0.591128   
LFR 200M       runtime            NaN                NaN   
               modularity         NaN                NaN   
com-amazon     runtime            NaN           1.024390   
               modularity         NaN           0.925548   
com-friendster runtime            NaN        5733.810000   
               modularity         NaN           0.623746   
com-lj         runtime     222.109561          96.720000   
               modularity    0.744079           0.747613   
com-orkut      runtime     217.506169         170.680000   
               modularity    0.664707           0.665134   
com-youtube    runtime            NaN           6.388920   
               modularity         NaN           0.718879   
hypercube      runtime            NaN        1537.810000   
               modularity         NaN           0.572864   
in-2004        runtime            NaN          15.064000   
               modularity         NaN           0.98

In [11]:
all_data = frames['clustering'] \
    .merge(frames['algorithm_run'], left_on='algorithm_run_id', right_index=True) \
    .merge(frames['program_run'], left_on='program_run_id', right_index=True) \
    .groupby(['algorithm', 'graph']).agg({ 'hosts': 'first', 'runtime': 'mean', 'modularity': 'mean'}) \
    .stack() \
    .unstack(['algorithm']).unstack()
    
all_data[('', 'hosts')] = all_data[('synchronous local moving', 'hosts')].astype(int)
all_data[('ground truth', 'modularity')] = frames['ground_truth']['modularity']

all_data = all_data \
    [[('ground truth', 'modularity'), ('sequential louvain', 'runtime'), ('sequential louvain', 'modularity'), 
      ('', 'hosts'),
      ('partitioned local moving', 'runtime'),
      ('partitioned local moving', 'modularity'),
      ('synchronous local moving', 'runtime'),
      ('synchronous local moving', 'modularity')]]

def my_round(x):
    if x:
        return round(x, 1)
    else: 
        return np.NaN
    
all_data[[('sequential louvain', 'runtime'), ('partitioned local moving', 'runtime'), ('synchronous local moving', 'runtime')]] = all_data.apply(lambda x: pd.Series([my_round(x[('sequential louvain', 'runtime')]), my_round(x[('partitioned local moving', 'runtime')]), my_round(x[('synchronous local moving', 'runtime')])]), axis=1)

all_data = all_data.loc[frames['program_run'].sort_values('edge_count')['graph'].dropna().unique()]

print(all_data.to_latex().replace('NaN', '   ').replace('runtime', 'runtime [s]'))

all_data

\begin{tabular}{lrrrrrrrr}
\toprule
algorithm & ground truth & \multicolumn{3}{l}{sequential louvain} & \multicolumn{2}{l}{partitioned local moving} & \multicolumn{2}{l}{synchronous local moving} \\
{} &   modularity &            runtime [s] & modularity & hosts &                  runtime [s] & modularity &                  runtime [s] & modularity \\
graph          &              &                    &            &       &                          &            &                          &            \\
\midrule
com-amazon     &              &                1.1 &   0.925990 &     2 &                          &            &                     35.5 &   0.925443 \\
com-youtube    &              &                6.3 &   0.719774 &     2 &                    150.6 &   0.713472 &                     41.2 &   0.720974 \\
LFR 100K       &     0.524983 &                1.5 &   0.521162 &     2 &                          &            &                      9.6 &   0.524983 \\
in-2004        & 

algorithm      ground truth sequential louvain                   \
                 modularity            runtime modularity hosts   
graph                                                             
com-amazon              NaN                1.1   0.925990     2   
com-youtube             NaN                6.3   0.719774     2   
LFR 100K           0.524983                1.5   0.521162     2   
in-2004                 NaN               14.6   0.980155     4   
com-lj                  NaN              102.3   0.748221     8   
hypercube               NaN             1534.8   0.572928     8   
com-orkut               NaN              173.7   0.667512     8   
LFR 1M             0.592803               29.2   0.591490     4   
uk-2002                 NaN              587.3   0.989749     8   
LFR 10M            0.599280             1021.0   0.599284     8   
com-friendster          NaN             5437.9   0.611236    16   
uk-2007-05              NaN                NaN        NaN    16   
LFR 100M           0.599928                NaN        NaN    16   
LFR 200M           0.599963                NaN        NaN    32   

algorithm      partitioned local moving            synchronous local moving  \
                                runtime modularity                  runtime   
graph                                                                         
com-amazon                          NaN        NaN                     35.5   
com-youtube                       150.6   0.713472                     41.2   
LFR 100K                            NaN        NaN                      9.6   
in-2004                            68.6   0.980307                     98.2   
com-lj                            222.1   0.744079                    167.7   
hypercube                           NaN        NaN                    115.4   
com-orkut                         217.5   0.664707                    132.8   
LFR 1M                             49.1   0.592719                     27.2   
uk-2002                           324.4   0.989725                    222.0   
LFR 10M                           225.2   0.599273                    151.7   
com-friendster                   1443.0   0.616655                   1597.7   
uk-2007-05                       1185.7   0.996166                    496.9   
LFR 100M                        24686.9   0.599892                    640.9   
LFR 200M                            NaN        NaN                    860.8   

algorithm                  
               modularity  
graph                      
com-amazon       0.925443  
com-youtube      0.720974  
LFR 100K         0.524983  
in-2004          0.980120  
com-lj           0.730967  
hypercube        0.539654  
com-orkut        0.657185  
LFR 1M           0.592534  
uk-2002          0.989648  
LFR 10M          0.596983  
com-friendster   0.612749  
uk-2007-05       0.996190  
LFR 100M         0.564620  
LFR 200M         0.574363